<a href="https://colab.research.google.com/github/1122131uhi/dataAnalytics/blob/master/Tutorial_3a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this tutorial we will train and test a linear regressor. This is something you have already done in part 3 of the course document. So, I will miss out all of the extra notes and just get to business.

In [1]:
# needed to create the data frame
import pandas as pd

# needed to help with speedy maths based calculations
import numpy as np

# create data frame from csv file we hosted on our github
df = pd.read_csv('https://raw.githubusercontent.com/1122131uhi/dataAnalytics/master/tutorial2lineardata.csv', index_col=0, )

In [2]:
# make sure we have our data by printing it out
print(df[:6])
# print(df) #all

    day  temp  wdsp  NUM_TRIPS
2     6  20.0   7.4   0.631126
3     7  30.0  10.8   0.724951
6     3  30.4   5.7   0.716044
8     5  37.9  16.4   0.799994
9     6  25.7  13.8   0.872611
10    7  21.6   6.9   0.809792


In [3]:
# needed to help with speedy maths based calculations
import numpy as np

# iloc allows us to select by rows. Here, we are shuffling the data by rows determined at random.
shuffle = df.iloc[np.random.permutation(len(df))]

# we are selecting all rows of the columns outliined i.e. The 3rd (2 as indexes start from 0)
predictors = shuffle.iloc[:,0:3]
# Since it is the last column, we can also use
# predictorTest = shuffle.iloc[:,-1]

# print out the first 6 rows of predictors.
print(predictors[:6])

      day  temp  wdsp
405     3  27.4   9.5
87      7  43.4   5.0
1087    6  47.4   4.9
372     5  27.1   9.4
252     4  61.6   7.2
1057    4  47.6   6.7


In [4]:
# print out the shuffled data (first 5 rows)
shuffle[:5]

,day,temp,wdsp,NUM_TRIPS
405,3,27.4,9.5,0.771915
87,7,43.4,5.0,0.860430
1087,6,47.4,4.9,0.799774
372,5,27.1,9.4,0.838513
252,4,61.6,7.2,0.818874


In [5]:
# Select all rows for the 2nd column (i.e. 1)
targets = shuffle.iloc[:,3]

# print out the first 6 rows of the targets data.
print(targets[:6])

405     0.771915
87      0.860430
1087    0.799774
372     0.838513
252     0.818874
1057    0.797393
Name: NUM_TRIPS, dtype: float64


In [6]:
# A scale is not required here, but the constant will be useful in the assignment.
SCALE_NUM_TRIPS = 1.0

In [7]:
# Split our data into a training set i.e. 80% of the length of the shuffle array
trainsize = int(len(shuffle['NUM_TRIPS'])*0.8)
# The test set size is 100% - 80% = 20% of the length of the shuffle array.
testsize = len(shuffle['NUM_TRIPS']) - trainsize

# Define the number of input values (predictors)
nppredictors = 3
# Define the number of output values (targets)
noutputs = 1

In [9]:
# import tensorflow
# %tensorflow_version 1.x
!pip install tensorflow==1.15.2
import tensorflow as tf

# check the version
print(tf.__version__)

# needed for high-level file management
import shutil  

# logging for tensorflow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.INFO)

# removes a saved model from the last training attempt.
shutil.rmtree('/tmp/linear_regression_trained_model', ignore_errors=True)

# This is the core of our linear regressor

# You can see that we save the model, use the the Adam optimization algorithm, which is an extension 
# to stochastic gradient descent that has recently seen broader adoption for deep learning applications 
# in computer vision and natural language processing and infer real valued columns from input which interprets 
# all inputs as dense, fixed-length float values.

# See the link for more information
# https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/contrib/learn/LinearRegressor
estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model', optimizer=tf.train.AdamOptimizer(learning_rate=0.1), enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(predictors.values)))

# Prints a log to show model is starting to train
print("starting to train");

# Train the model. Pass in predictor values and target values.
estimator.fit(predictors[:trainsize].values, targets[:trainsize].values.reshape(trainsize, noutputs)/SCALE_NUM_TRIPS, steps=10000)

# Next, we can check our predictions based on our predictors.
preds = estimator.predict(x=predictors[trainsize:].values)

# Apply the Scale value (not really needed here) to the outputs.
predslistscale = preds['scores']*SCALE_NUM_TRIPS

# pred = format(str(predslistscale)) # useful for checking outputs and printing.

# Calculate RMSE i.e. how good the model works using the predictions and targets.
# i.e. take the difference between the actual and the forecast then square the difference, 
# find the average of all the squares and then find the square root. 
# The RMSE essentially punishes larger errors i.e. it puts a heavier weight on larger errors.
rmse = np.sqrt(np.mean((targets[trainsize:].values - predslistscale)**2))
print('LinearRegression has RMSE of {0}'.format(rmse));


# Calculate the mean of the Number of Taxi Trips Values.
avg = np.mean(shuffle['NUM_TRIPS'][:trainsize])

# Calculate the RMSE using Number of Taxi Trips Values and the mean of all target values.
# The fit of a proposed regression model should therefore be better than the fit of the mean model.
# In this case, it doesn't seem to be the case but it will vary on every run.
rmse = np.sqrt(np.mean((shuffle['NUM_TRIPS'][trainsize:] - avg)**2))
print('Just using average = {0} has RMSE of {1}'.format(avg, rmse));

1.15.2
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f33448768d0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model', '_session_creation_timeout_secs': 7200}
starting to train
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Ru

In [10]:
input = pd.DataFrame.from_dict(data = 
				{'day' : [1,1,1],
         'temp' : [61.8, 31.2, 77.0],
         'wdsp' : [5.0, 3.0, 8.0]
        })
					

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)
# Assume number of trips scale value is 600000 when at a maximum, based on the analysis from Tutorial 2
predslistnorm = preds['scores']
predslistscale = preds['scores']*600000
prednorm = format(str(predslistnorm))
pred = format(str(predslistscale))
print(prednorm)
print(pred)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3344865978>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model/model.ck

From test inputs we have used, the first element in the array here is similar to this actual data:

"165",1,62.4,5.6,0.668458757342322

We can see that the temperature is slightly higher in the test data (which means less trips, but slightly higher wind means more trips. So, the difference between (actual) 0.668 and (predicted) 0.576 (rounded to 3 significant figures) seems reasonable. 

Similarly with the second:

"389",1,26.6,3.1,0.763954173062719, which has higher number of trips due to a lower temperature and also with a slightly higher wind speed.

And with the third:

"571",1,77.2,8.4,0.724652060408235

The last prediction with the higher temperature seems to punish the values more. 

In [11]:
input = pd.DataFrame.from_dict(data = 
				{'day' : [6,6,6],
         'temp' : [61.8, 31.2, 77.0],
         'wdsp' : [5.0, 3.0, 8.0]
        })
					

estimator = tf.contrib.learn.SKCompat(tf.contrib.learn.LinearRegressor(model_dir='/tmp/linear_regression_trained_model', enable_centered_bias=False, feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(input.values)))

preds = estimator.predict(x=input.values)
# Assume number of trips scale value is 600000 when at a maximum, based on the analysis from Tutorial 2
predslistnorm = preds['scores']
predslistscale = preds['scores']*600000
prednorm = format(str(predslistnorm))
pred = format(str(predslistscale))
print(prednorm)
print(pred)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3342754278>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/linear_regression_trained_model', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/linear_regression_trained_model/model.ck

This test uses day 6 (Friday) instead of day 1 (Sunday) which shows higher number of trips. The other values were left the same.

Things to think about for the assignment. Make a validation set i.e. 5% of the data (or maybe more). This should be used for this type of testing. My values are simply made up.

You should also remember to use different models with different data. In this case, I would maybe take each input valuable separately and make a regression model for each, then different variations i.e. any 2.

Remember, you need to write up your results in the assignment. 